In [1]:
from astropy.table import Table  
import numpy as np

data_table = Table.read('/home/douglas/Gala-code/correctactions.fits', format = 'fits') 
print(data_table[1].columns)

<TableColumns names=('idx','Jr','Jphi','Jz','Theta_r','Theta_phi','Theta_z','Freq_r','Freq_phi','Freq_z')>


In [2]:
marchetti = Table.read('/home/douglas/data.fits', format = 'fits')
print(marchetti[1].columns)

<TableColumns names=('source_id','ra','dec','parallax','e_parallax','pmra','e_pmra','pmdec','e_pmdec','vrad','e_vrad','GMag','dist','el_dist','eu_dist','rGC','el_rGC','eu_rGC','RGC','el_RGC','eu_RGC','xGC','el_xGC','eu_xGC','yGC','el_yGC','eu_yGC','zGC','el_zGC','eu_zGC','U','el_U','eu_U','V','el_V','eu_V','W','el_W','eu_W','UW','el_UW','eu_UW','vR','el_vR','eu_vR','vtot','el_vtot','eu_vtot','P_ub')>


In [3]:
Jr = np.array(data_table["Jr"][:119629])
Jphi = np.array(data_table["Jphi"][:119629])
Jz = np.array(data_table["Jz"][:119629])
Or = np.array(data_table["Freq_r"][:119629])
Ophi = np.array(data_table["Freq_phi"][:119629])
Oz = np.array(data_table["Freq_z"][:119629])
idx = np.array(data_table["idx"][:119629])

In [4]:
X = []
Y = []
Z = []
U = []
V = []
W = []

for i in idx:
    X.append(marchetti["xGC"][i]*0.001)
    Y.append(marchetti["yGC"][i]*0.001)
    Z.append(marchetti["zGC"][i]*0.001)
    U.append(marchetti["U"][i])
    V.append(marchetti["V"][i])
    W.append(marchetti["W"][i])

X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)
U = np.array(U)
V = np.array(V)
W = np.array(W)

In [120]:
zmax = []
maxr = []
minr = []
import astropy.units as u

for i in range(0,len(Jr)):
    #Defining Phase Space
    phase = gd.PhaseSpacePosition(pos=[X[i], Y[i], Z[i]] *u.kpc,
                            vel=[U[i], V[i], W[i]] *u.km/u.s)
    
    #Integrating orbit
    orb = gp.Hamiltonian(pot).integrate_orbit(phase, dt=0.1, n_steps=10000)
    
    zmax.append(orb.zmax().value)
    maxr.append(orb.apocenter().value)
    minr.append(orb.pericenter().value)

In [124]:
from astropy.io import fits
c1 = fits.Column(name='zmax', array=np.array(zmax), format='D', unit = 'kpc')
c2 = fits.Column(name='maxr', array=np.array(maxr), format='D', unit = 'kpc')
c3 = fits.Column(name='minr', array=np.array(minr), format='D', unit = 'kpc')

t = fits.BinTableHDU.from_columns([c1, c2, c3])

t.writeto('correctorbitvalues.fits')